In [1]:
import time
import pandas as pd
import numpy as np
from tqdm import tqdm
import os
import utils # written by author
from glob import glob
from datetime import datetime, timedelta
import multiprocessing as mp
import gc # for automatic releasing memory

In [4]:
##################################################
# Load transaction 
##################################################
col = ['msno','transaction_date','membership_expire_date','is_cancel']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col)

transactions = transactions.head(n = 5000)
##################################################
# Convert string to datetime format
##################################################
transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))


100%|██████████| 4/4 [00:29<00:00,  7.30s/it]


# 以下沒在官方文件發現, 是自己推斷（切記過分推斷, 怪異的都做noise不考慮)

    # 1.如果可以當作預測的order發生兩次, 以最後一筆order當作判斷label的依據

    # 2.如果剛好發生在check points(等號)貌似是churn
    
#Note:


In [10]:
def filter_with_window_size(x, w = 0):
    # w: window_size(short_window_back)
    if w == 0:
        # for data cleaning: remove weird label 
        # 找出這個人在17年二月份中的有到期的交易紀錄
        if x[(x.membership_expire_date >= datetime.strptime('2017-02-01', '%Y-%m-%d'))&(x.membership_expire_date <= datetime.strptime('2017-02-28', '%Y-%m-%d'))].empty:
            # 如果這個交易紀錄是空的,把它濾掉
            x['key'] = 1 # key = 1, 濾掉
        else:
            x['key'] = 0
            x['w'] = w # # 代表這群人將會被預測在17年三月是否流失(true label is provided by train_v2)
        return x
    elif w == 1:
        # 找出這個人在17年一月份中的有到期的交易紀錄
        if x[(x.membership_expire_date >= datetime.strptime('2017-01-01', '%Y-%m-%d'))&(x.membership_expire_date <= datetime.strptime('2017-01-31', '%Y-%m-%d'))].empty:
            # 如果這個交易紀錄是空的,把它濾掉
            x['key'] = 1 # key = 1, 濾掉
        else:
            x['key'] = 0
        return x
    elif w == 2:
        # 找出這個人在16年十二月份中的有到期的交易紀錄
        if x[(x.membership_expire_date >= datetime.strptime('2016-12-01', '%Y-%m-%d'))&(x.membership_expire_date <= datetime.strptime('2016-12-31', '%Y-%m-%d'))].empty:
            # 如果這個交易紀錄是空的,把它濾掉
            x['key'] = 1 # key = 1, 濾掉
        else:
            x['key'] = 0
        return x
def label_generation(x, w):   
    if w == 1:
        # 1. 先判斷哪一筆訂單的到期日是落17年在1月中
        df = x[(x.membership_expire_date >= datetime.strptime('2017-01-01', '%Y-%m-%d')  )
                         &(x.membership_expire_date <= datetime.strptime('2017-01-31', '%Y-%m-%d')  )
                         ].tail(n = 1) #如果df有兩筆,取最後一筆
        # 2.find check point
        df['check_point'] = df.membership_expire_date + timedelta(30)
        # 3.找出哪一筆訂單開始在scope of prediction(2月)的order會被拿來當作判斷是否churn的依據
        future = x[(x.transaction_date >= datetime.strptime('2017-02-01', '%Y-%m-%d'))
                   &(x.transaction_date <= datetime.strptime('2017-02-28', '%Y-%m-%d'))
                  ].tail(n = 1)  
        # 4. 判斷is_churn
        if future.empty:
            pass
        else:
            if future.transaction_date.iloc[0] < df.check_point.iloc[0] and future.is_cancel.iloc[0] == 0:
                x['is_churn'] = 0
            else:
                x['is_churn'] = 1
        return x
    elif w == 2:
        # 1. 先判斷哪一筆訂單的到期日是落在16年12月中
        df = x[(x.membership_expire_date >= datetime.strptime('2016-12-01', '%Y-%m-%d')  )
                     &(x.membership_expire_date <= datetime.strptime('2016-12-31', '%Y-%m-%d')  )
                     ].tail(n = 1) 
        # 2.find check point
        df['check_point'] = df.membership_expire_date + timedelta(30)
        # 3.找出哪一筆訂單開始在scope of prediction(17年1月)的order會被拿來當作判斷是否churn的依據
        future = x[(x.transaction_date >= datetime.strptime('2017-01-01', '%Y-%m-%d'))
                   &(x.transaction_date <= datetime.strptime('2017-01-31', '%Y-%m-%d'))
                  ].tail(n = 1)  
        # 4. 判斷is_churn
        if future.empty:
            pass
        else:
            if future.transaction_date.iloc[0] < df.check_point.iloc[0] and future.is_cancel.iloc[0] == 0:
                x['is_churn'] = 0
            else:
                x['is_churn'] = 1
        return x


# w = 0

# output

In [22]:
d = train.groupby('msno').apply(filter_with_window_size, w = 0)
output_col = ['msno','is_churn','w']
d = d[d.key != 1][output_col].reset_index(drop=True).to_csv('../../input/preprocessed_data/trainW-0.csv', index = 'False')


In [32]:
d = train.groupby('msno').apply(filter_with_window_size, w = 0)
output_col = ['msno','is_churn','w']
d = d[d.key != 1][output_col]
d = d.drop_duplicates('msno', keep = 'last') #  因為前面有跟transactions merge起來
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 81609 to 946911
Data columns (total 3 columns):
msno        22 non-null object
is_churn    22 non-null int64
w           22 non-null float64
dtypes: float64(1), int64(1), object(1)
memory usage: 704.0+ bytes


# w = 1

In [47]:
transactions[transactions.msno == 'zyGHNWQXe5eyHc3r5iuPPfRKDesYh4/WZi3Jxzp9STM=']

,msno,transaction_date,membership_expire_date,is_cancel
5732337,zyGHNWQXe5eyHc3r5iuPPfRKDesYh4/WZi3Jxzp9STM=,2017-01-13,2017-02-13,0


In [12]:
T = 1
output_col = ['msno','is_churn']
generate_label_col = ['msno','transaction_date','membership_expire_date','is_cancel']

df = transactions.groupby('msno').apply(filter_with_window_size, w = 1)
df = df[df.key != 1][generate_label_col].reset_index(drop=True).groupby('msno').apply(label_generation, w = 1)[output_col].dropna()
df = df.drop_duplicates('msno', keep = 'last')
df['w'] = T # w = 1代表這群人將會被預測在17年二月是否流失(true label is generated by below function)        
print ('shape of trainW-{0}'.format(T), df.shape)


shape of trainW-1 (197, 3)


In [25]:
d.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 379 entries, 0 to 378
Data columns (total 6 columns):
is_cancel                 379 non-null int64
is_churn                  375 non-null float64
membership_expire_date    379 non-null datetime64[ns]
msno                      379 non-null object
transaction_date          379 non-null datetime64[ns]
w                         379 non-null float64
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 17.8+ KB


In [31]:
d = d.drop_duplicates('msno', keep = 'first')
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23 entries, 1 to 378
Data columns (total 6 columns):
is_cancel                 23 non-null int64
is_churn                  21 non-null float64
membership_expire_date    23 non-null datetime64[ns]
msno                      23 non-null object
transaction_date          23 non-null datetime64[ns]
w                         23 non-null float64
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 1.3+ KB


# w = 2

In [34]:
d = transactions.groupby('msno').apply(filter_with_window_size, w = 2)
generate_label_col = ['msno','transaction_date','membership_expire_date','is_cancel','w']
d = d[d.key != 1][generate_label_col].reset_index(drop=True) # 省記憶體(index也是一個欄位)

d =d.groupby('msno').apply(label_generation, w = 2)
output_col = ['msno','is_churn','w']


In [36]:
d = d.drop_duplicates('msno', keep = 'last')
d.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 25 entries, 0 to 357
Data columns (total 6 columns):
is_cancel                 25 non-null int64
is_churn                  20 non-null float64
membership_expire_date    25 non-null datetime64[ns]
msno                      25 non-null object
transaction_date          25 non-null datetime64[ns]
w                         25 non-null float64
dtypes: datetime64[ns](2), float64(2), int64(1), object(1)
memory usage: 1.4+ KB


In [ ]:
for i in range(2)

In [210]:
# 1. parallel processing
def make(T):
    output_col = ['msno','is_churn','w']
    generate_label_col = ['msno','transaction_date','membership_expire_date','is_cancel','w']
    if T == 0:
        df = train.groupby('msno').apply(filter_with_window_size, w = 0)
        df[df.key != 1][output_col].reset_index(drop=True).to_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T), index = 'False')
        gc.collect()
    elif T == 1:
        df = transactions.groupby('msno').apply(filter_with_window_size, w = 1)
        df = df[df.key != 1][generate_label_col].reset_index(drop=True) # 省記憶體(index也是一個欄位)
        df.groupby('msno').apply(label_generation, w = 1)[output_col].dropna().to_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T), index = 'False')
        gc.collect()    
    elif T == 2:
        df = transactions.groupby('msno').apply(filter_with_window_size, w = 2)
        df = df[df.key != 1][generate_label_col].reset_index(drop=True) # 省記憶體(index也是一個欄位)
        df.groupby('msno').apply(label_generation, w = 2)[output_col].dropna().to_csv('../input/preprocessed_data/trainW-{0}.csv'.format(T), index = 'False')
        gc.collect()

        

In [211]:
# multi processing
mp_pool = mp.Pool(4)
mp_pool.map(make, [0,1,2])

[None, None, None]

In [ ]:
# 找出二月份(future)中沒有交易的人且也
d = transactions[(transactions.transaction_date >= datetime.strptime('2017-03-01', '%Y-%m-%d') )
                 &(transactions.transaction_date <=datetime.strptime('2017-04-30', '%Y-%m-%d') )
                 ] #1.找出在三月份交易的人

In [4]:
##################################################
# Load transaction 
##################################################
col = ['msno','transaction_date','membership_expire_date','is_cancel']
transactions = utils.read_multiple_csv('../../input/preprocessed_data/transactions', col)

# ##################################################
# # Convert string to datetime format
# ##################################################
# transactions['membership_expire_date']  = transactions.membership_expire_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))
# transactions['transaction_date']  = transactions.transaction_date.apply(lambda x: datetime.strptime(x, '%Y-%m-%d'))

##################################################
# Load train set user and test set user
##################################################
train = pd.read_csv('../../input/train_v2.csv') #只用這個檔案, 然後做data augmentation
#test = pd.read_csv('../input/sample_submission_v2.csv')




100%|██████████| 4/4 [00:31<00:00,  7.75s/it]


In [5]:
train_ = pd.merge(train, transactions, on = 'msno', how = 'left')
train_.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16245636 entries, 0 to 16245635
Data columns (total 5 columns):
msno                      object
is_churn                  int64
transaction_date          object
membership_expire_date    object
is_cancel                 int64
dtypes: int64(2), object(3)
memory usage: 743.7+ MB


In [6]:
train = pd.merge(train, transactions, on = 'msno', how = 'left')
train = train.dropna()
train.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16245636 entries, 0 to 16245635
Data columns (total 5 columns):
msno                      object
is_churn                  int64
transaction_date          object
membership_expire_date    object
is_cancel                 int64
dtypes: int64(2), object(3)
memory usage: 743.7+ MB
